In [6]:
from torchvision.transforms.transforms import ToTensor
import torch 
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import time
import math
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_ds= torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_ds= torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size=128, shuffle=True)
test_dl = torch.utils.data.DataLoader(dataset=test_ds, batch_size=128, shuffle=False)

examples = iter(test_dl)
example_images, example_targets = examples.next()

# for i in range(8):
#   plt.subplot(2,4,i+1)
#   plt.imshow(example_images[i][0], cmap='gray')
# plt.show()

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.fc1 = nn.Linear(28 * 28, 100)
    self.fc2 = nn.Linear(100, 100)
    self.fc3 = nn.Linear(100, 10)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    x = x.view(-1, 28 * 28)
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = torch.relu(self.fc2(x))
    x = self.dropout(x)
    x = self.fc3(x)
    return x

model = NeuralNetwork().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001, betas=[0.9, 0.999])

n_epochs = 10

model.train()
tic_total_train = time.time()
epoch_times = []

for epoch in range(n_epochs):
   
    train_loss = 0.0
    tic_train_epoch = time.time()

    
    for data, target in train_dl:
       
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = criterion(output, target)
       
        loss.backward()
        
        optimizer.step()
       
        train_loss += loss.item()*data.size(0)
             
    train_loss = train_loss/len(train_dl.dataset)
    toc_train_epoch = time.time()
    time_per_epoch = toc_train_epoch - tic_train_epoch
    epoch_times.append(time_per_epoch)

    print('Epoch: {} \tTraining Loss: {:.6f} \tEpoch Time: {:.2f}s'.format(
        epoch+1, 
        train_loss,
        time_per_epoch,
        ))

print("\nAverage Epoch Training Time: {:.2f}s".format(np.average(epoch_times)))

print("-------------------------------------------------------------------------")
    
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()

tic_eval = time.time()
for data, target in test_dl:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # getting the current batch size
    batch_size = data.size(0)
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1       
toc_eval = time.time()

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i+1), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

# calculate and print avg test loss
test_loss = test_loss/len(test_dl.dataset)
print('\nAverage Test Loss(Model Evaluation): {:.6f}\n'.format(test_loss))

print('Test Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

num_eval_batches = math.ceil(len(test_dl)/128)

average_batch_inference_time = (1000* (toc_eval - tic_eval) / num_eval_batches) # in ms

print("\nAverage Batch Inference Time: {:.2f}ms".format(average_batch_inference_time))

toc_total_train = time.time()
total_train_time = toc_total_train - tic_total_train

print("\nTotal Training Time: {:.2f}s".format(total_train_time))





  





Epoch: 1 	Training Loss: 0.492628 	Epoch Time: 6.26s
Epoch: 2 	Training Loss: 0.210983 	Epoch Time: 6.21s
Epoch: 3 	Training Loss: 0.160320 	Epoch Time: 6.19s
Epoch: 4 	Training Loss: 0.137065 	Epoch Time: 6.23s
Epoch: 5 	Training Loss: 0.117955 	Epoch Time: 6.34s
Epoch: 6 	Training Loss: 0.105298 	Epoch Time: 6.22s
Epoch: 7 	Training Loss: 0.092336 	Epoch Time: 6.20s
Epoch: 8 	Training Loss: 0.086436 	Epoch Time: 6.20s
Epoch: 9 	Training Loss: 0.079494 	Epoch Time: 6.24s
Epoch: 10 	Training Loss: 0.075308 	Epoch Time: 6.27s

Average Epoch Training Time: 6.24s
-------------------------------------------------------------------------
Test Accuracy of     1: 98% (968/980)
Test Accuracy of     2: 98% (1121/1135)
Test Accuracy of     3: 98% (1013/1032)
Test Accuracy of     4: 97% (988/1010)
Test Accuracy of     5: 98% (967/982)
Test Accuracy of     6: 96% (860/892)
Test Accuracy of     7: 98% (939/958)
Test Accuracy of     8: 96% (991/1028)
Test Accuracy of     9: 97% (945/974)
Test Accura

In [8]:
# export JSON
metrics = {
    "model_name": "MLP",
    "framework_name": "PyTorch",
    "dataset": "MNIST Digits",
    "task": "classification",
    "total_training_time": total_train_time,         # in seconds
    "average_epoch_training_time": np.average(epoch_times),  # in seconds
    "average_batch_inference_time": average_batch_inference_time,  # in milliseconds
    "final_training_loss": test_loss,
    "final_evaluation_accuracy": 100. * np.sum(class_correct) / np.sum(class_total),
}

with open("m1-pytorch-mlp.json", "w") as outfile:
  json.dump(metrics, outfile)


In [9]:
metrics

{'model_name': 'MLP',
 'framework_name': 'PyTorch',
 'dataset': 'MNIST Digits',
 'task': 'classification',
 'total_training_time': 63.25790095329285,
 'average_epoch_training_time': 6.235158848762512,
 'average_batch_inference_time': 895.6584930419922,
 'final_training_loss': 0.0791693767288234,
 'final_evaluation_accuracy': 97.69}